# Problem set 2
Before we start working on some exercises we will briefly introduce two concepts in Python. First, importing and exporting data. Second, using functions. If you are already familiar
with these features, you can skip the next two sections and jump directly to the exercises.

First, import all necessary packages. We have made a .py file that we will use as a "toolbox". We will fill this toolbox with functions, that we will use as we progress through the course. Exactly how you structure this toolbox is up to you (if you i.e. want to turn it into a class).

In [3]:
import numpy as np
from numpy import linalg as la
import pandas as pd
from io import StringIO
from tabulate import tabulate
from matplotlib import pyplot as plt

#Supress Future Warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Import this weeks LinearModels.py file
import w2_LinearModels_ante as lm
%load_ext autoreload
%autoreload 2

## Importing and exporting data in Python
The easiest way to import data into an numpy array is using a .txt file. Normally we specify a path to the text file, but we will create a fake one to illustrate.

Sadly, there is no direct way to load an excel sheet into numpy. The easiest solution is to use pandas as an intermediate.

### Exporting Data
To save a numpy array as a .txt file is easy:

*If one has large numpy arrays and wants to store them efficiently, they can be saved as a binary .npy files. Such files are not compatible with other programs.*

## Exercises 
### Import data

The exercise takes up the union membership example from the cover sheet. The data set WAGEPAN.TXT contains information about 545 men who worked every year from 1980 to 1987 in the US. The variables of interest are


| Variable | Content |
|-|-|
| nr | Variable that identifies the individual  |
| year | Year of observation |
| Black | Black |
| Exper | Years since left school |
| Hisp | Hispanic |
| Married | Marital status |
| Educ | Years of schooling |
| Union | Union membership |
| Lwage | Natural logarithm of hourly wages |
| Expersq | Exper2 |

Consider the following wage equation:

$$
\begin{align}
\log\left(wage_{it}\right) & =\beta_{0}+\beta_{1}\textit{exper}_{it}+\beta_{2}\textit{exper}_{it}^{2}+\beta_{3}\textit{union}_{it}+\beta_{4}\textit{married}_{i} +\beta_{5}\textit{educ}_{i}+\beta_{6}\textit{hisp}_{i}+\beta_{7}\textit{black}_{i}+c_{i}+u_{it} \tag{1}
\end{align}
$$

Note that *educ*, *hisp*, and *black* are time-invariant variables.

The data has 10 columns. Named from 0 to 9. Here is a variable describtion:
- Column 0: ID
- Column 1: Year
- Column 2: Black
- Column 3: Experience
- Column 4: Hispanic
- Column 5: Married
- Column 6: Education
- Column 7: Union
- Column 8: ln wage
- Column 9: Experience sqr

Start by loading the data. Some of this has been done for you already. Since we are working with panels, we need to know how many persons there are and how many time periods we observe them. Since we operate using a balanced panel, this makes our life a little easier.

In [4]:
# First, import the data into numpy. 
# Data should load the wagepan.txt file.
data = np.loadtxt('wagepan.txt', delimiter=',', skiprows=0)
id_array = np.array(data[:, 0])

# Count how many persons we have. This returns a tuple with the unique IDs,
# and the number of times each person is observed.
unique_id = np.unique(id_array, return_counts=True)
N = unique_id[0].size
T = int(unique_id[1].mean())
year = np.array(data[:, 1], dtype=int)

In [5]:
# Load the rest of the data into arrays.
y = data[:, 8]

# x needs to have a constant vector in the first row. How would you add this? 
# Note that the order is set to match the order of variables in the model.
x =  np.array([
    np.ones(data.shape[0]),  # Constant term
    data[:, 3],
    data[:, 9],
    data[:, 7],
    data[:, 5],
    data[:, 6],
    data[:, 4],
    data[:, 2]
]).T

# Lets also make some variable names
label_y = 'Log wage'
label_x = [
    'Constant', 
    'Experience', 
    'Experience sqr', 
    'Union',
    'Married', 
    'Education', 
    'Hispanic', 
    'Black', 
]

## Pooled OLS (POLS) Estimator
- **Estimate (1) by pooled OLS,** thus considering for the moment the unobserved components of (q) as one (composite) error term $v_{it}=c_{i}+u_{it}$. 
- Fill in the remaining parts of the function est_ols() in the accompanying python file (LinearModelsWeek2_ante.py) to estimate the model.
- What assumptions are made about $E\left[c_{i}\mathbf{x}_{it}\right]$ and $E\left[u_{it}\mathbf{x}_{it}\right]$ when justifying this estimation approach? (Hint: See Wooldridge p. 283)

In [9]:
# Estimate coefficients using the estimate function
ols_result = lm.estimate(y, x, N=N, T=T)
# Print the results
for label, b_k in zip(label_x, np.ravel(ols_result['b_hat'])):
    print(f'{label:16}: {b_k:.4f}')

Constant        : -0.0347
Experience      : 0.0892
Experience sqr  : -0.0028
Union           : 0.1801
Married         : 0.1077
Education       : 0.0994
Hispanic        : 0.0157
Black           : -0.1438


Calculate the standard errors of the coefficients. This is very similar to previous week's exercise. (Hint: See Wooldridge p. 59-60)

In [11]:
# Calculate the residuals
resid = y - x @ ols_result['b_hat']

# Calculate estimate of variance of residuals
SSR = np.sum(resid**2)
K = x.shape[1]
sigma = SSR / (N - K)

# Calculate the variance-covariance matrix
cov = sigma * np.linalg.inv(x.T @ x)

# Calculate the standard errors 
# Make sure to output the result in a vector
se = np.sqrt(np.diag(cov))

#Print results
for label, b_k in zip(label_x, np.ravel(ols_result['b_hat'])):
    print(f'{label:16}: {b_k:.4f}')

Constant        : -0.0347
Experience      : 0.0892
Experience sqr  : -0.0028
Union           : 0.1801
Married         : 0.1077
Education       : 0.0994
Hispanic        : 0.0157
Black           : -0.1438


Fill in the functions estimate() and variance() in the accompanying python file. You can reuse most of the code above.

Using the function, print_table(), you should reproduce the table below.

In [12]:
# Estimate model using OLS
ols_result = lm.estimate(y,x, N=N, T=T)

# Print table
lm.print_table((label_y, label_x), ols_result, title="Pooled OLS", floatfmt='.4f')

Pooled OLS
Dependent variable: Log wage

                   Beta       Se    t-values
--------------  -------  -------  ----------
Constant        -0.0347   0.4807     -0.0722
Experience       0.0892   3.4138      0.0261
Experience sqr  -0.0028  31.1759     -0.0001
Union            0.1801   0.2375      0.7582
Married          0.1077   0.3185      0.3380
Education        0.0994   5.7188      0.0174
Hispanic         0.0157   0.1899      0.0827
Black           -0.1438   0.1635     -0.8800
R² = 0.187
σ² = 0.231


Pooled OLS <br>
Dependent variable: Log wage <br>

|                |    Beta |     Se |   t-values |
|----------------|---------|--------|------------|
| Constant       | -0.0347 | 0.0646 |    -0.5375 |
| Experience     |  0.0892 | 0.0101 |     8.8200 |
| Experience sqr | -0.0028 | 0.0007 |    -4.0272 |
| Union          |  0.1801 | 0.0171 |    10.5179 |
| Married        |  0.1077 | 0.0157 |     6.8592 |
| Education      |  0.0994 | 0.0047 |    21.2476 |
| Hispanic       |  0.0157 | 0.0208 |     0.7543 |
| Black          | -0.1438 | 0.0236 |    -6.1055 |
R² = 0.187 <br>
σ² = 0.231

## Fixed Effects (FE) Estimator
In the next step, we will estimate the model using fixed effects. This is done by first performing the fixed effects (within-groups) transformation on the data and then using pooled OLS on the transformed data.
We will break this down into multiple steps.

### Using numpy
Create a transformation matrix with dimensions $T \times T$ that can be used to transform the data. Note that the matrix will be premultiplied on the data for each individual, so the dimensions will match in the end.

In [13]:
# Create transformation matrix
def demeaning_matrix(T):
    Q_T = np.eye(T) - np.ones((T, T)) / T
    return Q_T

# Print the matrix
Q_T = demeaning_matrix(T)
print(f'Demeaning matrix for T={T} \n', Q_T)

Demeaning matrix for T=8 
 [[ 0.875 -0.125 -0.125 -0.125 -0.125 -0.125 -0.125 -0.125]
 [-0.125  0.875 -0.125 -0.125 -0.125 -0.125 -0.125 -0.125]
 [-0.125 -0.125  0.875 -0.125 -0.125 -0.125 -0.125 -0.125]
 [-0.125 -0.125 -0.125  0.875 -0.125 -0.125 -0.125 -0.125]
 [-0.125 -0.125 -0.125 -0.125  0.875 -0.125 -0.125 -0.125]
 [-0.125 -0.125 -0.125 -0.125 -0.125  0.875 -0.125 -0.125]
 [-0.125 -0.125 -0.125 -0.125 -0.125 -0.125  0.875 -0.125]
 [-0.125 -0.125 -0.125 -0.125 -0.125 -0.125 -0.125  0.875]]


Use the supplied perm() function to apply the transformation to the data.

In [19]:
# Transform the data (within / FE)
# Make sure y is (N*T, 1)
y_demean = lm.perm(Q_T, y.reshape(-1, 1))
x_demean = lm.perm(Q_T, x)

#print x_demean
print(x_demean)

[[  0.   -3.5 -24.5 ...   0.    0.    0. ]
 [  0.   -2.5 -21.5 ...   0.    0.    0. ]
 [  0.   -1.5 -16.5 ...   0.    0.    0. ]
 ...
 [  0.    1.5  22.5 ...   0.    0.    0. ]
 [  0.    2.5  43.5 ...   0.    0.    0. ]
 [  0.    3.5  66.5 ...   0.    0.    0. ]]


What is the rank and eigenvalues of the within transformed $\mathbf{X}$ matrix? Why?

What happens to *educ, hisp, and black* and the constant when the data are within transformed? 

In [20]:
# Create function to check rank of demeaned matrix, and return its eigenvalues.
def check_rank(x):
    print(f'Rank of demeaned x: {la.matrix_rank(x)}')
    lambdas, V = la.eig(x.T@x)
    np.set_printoptions(suppress=True)  # This is just to print nicely.
    print(f'Eigenvalues of within-transformed x: {lambdas.round(decimals=0)}')

# Check rank of demeaned x
check_rank(x_demean)

Rank of demeaned x: 4
Eigenvalues of within-transformed x: [4248875.    1871.     365.     329.       0.       0.       0.       0.]


Adjust `x_demean` such that the model can be estimated using the FE estimator. Adjust the labels to match with `x_demean`.

Estimate the model using the estimate() function, and print the results.

In [42]:
# Choose variables to include in fixed effects model
x_demean = x_demean[:, [0,1,2,3]]  # keep experience, experience^2, union, married
label_x_fe = [
    
    'Experience',
    'Experience sqr',
    'Union',
    'Married'
]

In [48]:
print("x_dot shape:", x_dot.shape)
print("Rank of x_dot:", la.matrix_rank(x_dot))

x_dot shape: (4360, 4)
Rank of x_dot: 3


In [49]:
# Column standard deviations
print("Std of columns:", np.std(x_dot, axis=0))

# Correlation matrix
print("Correlation matrix:\n", np.corrcoef(x_dot.T))


Std of columns: [31.13952767  0.27594708  0.32357663  0.        ]
Correlation matrix:
 [[ 1.         -0.02428942  0.37688252         nan]
 [-0.02428942  1.          0.01637536         nan]
 [ 0.37688252  0.01637536  1.                 nan]
 [        nan         nan         nan         nan]]


/Users/juliusvikjaer/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/juliusvikjaer/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [51]:
# Select variables for FE regression
# Drop constant and time-invariant regressors
y_dot = y_demean
x_dot = x_demean[:, [0, 1, 2]]  # Experience, Experience^2, Union

label_x_fe = [
    'Experience',
    'Experience sqr',
    'Union'
]

# Estimate FE OLS using the demeaned variables
fe_result = lm.estimate(y_dot, x_dot, transform='fe', N=N, T=T)

# Print results
lm.print_table((label_y, label_x_fe), fe_result, title='FE regression', floatfmt='.4f')


FE regression
Dependent variable: Log wage

                  Beta       Se  t-values
--------------  ------  -------  ----------------------------------
Experience      0.0037  11.2094  [0.00033    0.03723899 0.03175752]
Experience sqr  0.0828   0.0993  [0.00738329 0.83317513 0.71053413]
Union           0.1073   0.1165  [0.00957612 1.08062721 0.92156198]
R² = 0.137
σ² = 0.130


In [52]:
# Estimate FE OLS using the demeaned variables.
fe_result = lm.estimate(y_demean, x_demean, transform='fe', N=N, T=T)

# Print results
lm.print_table((label_y, label_x_fe), fe_result, title='FE regression', floatfmt='.4f')

LinAlgError: Singular matrix

You should get a table that looks like this:

FE regression<br>
Dependent variable: Log wage

|                |    Beta |     Se |   t-values |
|----------------|---------|--------|------------|
| Experience     |  0.1168 | 0.0084 |    13.8778 |
| Experience sqr | -0.0043 | 0.0006 |    -7.1057 |
| Union          |  0.0821 | 0.0193 |     4.2553 |
| Married        |  0.0453 | 0.0183 |     2.4743 |
R² = 0.178 <br>
σ² = 0.123

**NB:** Did you use the right standard errors? If not, implement the correct standard errors in the variance() function, and try again. (Hint: see Wooldridge p.306)

How big is the union premium according to the estimate from the FE model? Compare this with the estimate that you calculated from the pooled OLS regression. What does this suggest about $E\left[union_{it}c_{i}\right]$?

### Using pandas
An alternative to the perm function is to use panda dataframes to group the data. This has been done for the *entire* dataset below.

In [53]:
# load the data using pandas
pddat = pd.read_csv('wagepan.txt', delimiter=",", header=None)
pddat.columns = ["ID", "year", "black", "exp", "hisp", "mar", "educ", "union", "logwage", "expsq"]

# Rearrange the data
desired_order = ["ID", "year", "logwage", "exp", "expsq", "union", "mar", "educ", "hisp", "black"]
pddat = pddat[desired_order] # reorder columns

# Sort the data (it's important to ensure years are sorted correctly when transforming the data)
pddat = pddat.sort_values(["ID", "year"])

In [54]:
# demean data
pddat_demean = pddat - pddat.groupby("ID").transform('mean')

# reorder columns
# The order changes back to the original order after the transformation, due to the way pandas works.
pddat_demean = pddat_demean[desired_order] 

# turn into numpy array
datdemean = pddat_demean.to_numpy()

Look at the data. Select the variables that you need to estimate the model using FE. Estimate the model. Did you get the same results?

In [55]:
# Look at data
pddat_demean

,ID,year,logwage,exp,expsq,union,mar,educ,hisp,black
0,NaN,-3.5,-0.058112,-3.5,-24.5,-0.125,0.000,0.0,0.0,0.0
1,NaN,-2.5,0.597408,-2.5,-21.5,0.875,0.000,0.0,0.0,0.0
2,NaN,-1.5,0.088810,-1.5,-16.5,-0.125,0.000,0.0,0.0,0.0
3,NaN,-0.5,0.177561,-0.5,-9.5,-0.125,0.000,0.0,0.0,0.0
4,NaN,0.5,0.312473,0.5,-0.5,-0.125,0.000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
4355,NaN,-0.5,0.209697,-0.5,-13.5,-0.375,0.375,0.0,0.0,0.0
4356,NaN,0.5,-0.169639,0.5,3.5,0.625,0.375,0.0,0.0,0.0
4357,NaN,1.5,0.383780,1.5,22.5,-0.375,0.375,0.0,0.0,0.0
4358,NaN,2.5,0.363713,2.5,43.5,0.625,0.375,0.0,0.0,0.0


In [56]:
#Select variables
# Turn the data into numpy arrays
y_dot = y_demean
x_dot = x_demean[:, [0, 1, 2]]  # Experience, Experience^2, Union

In [57]:
# Estimate using the demeaned variables, y_dot and x_dot
fe_result = lm.estimate(y_dot, x_dot, transform='fe', N=N, T=T)

# Print results
lm.print_table((label_y, label_x_fe), fe_result, title='FE regression', floatfmt='.4f')

FE regression
Dependent variable: Log wage

                  Beta       Se  t-values
--------------  ------  -------  ----------------------------------
Experience      0.0037  11.2094  [0.00033    0.03723899 0.03175752]
Experience sqr  0.0828   0.0993  [0.00738329 0.83317513 0.71053413]
Union           0.1073   0.1165  [0.00957612 1.08062721 0.92156198]
R² = 0.137
σ² = 0.130


## First-difference (FD) Estimator
Construct $\mathbf{D}$ and use the procedure `perm` $(\mathbf{D},\mathbf{x})$ to compute first differences of the elements of $\mathbf{y}$ and $\mathbf{x}$. $\mathbf{D}$ should be a $(T-1) \times T$ matrix. Why?

What happens to *educ, hisp* and *black* and the constant when the data are transformed into first differences? What is the rank of the first differenced $\mathbf{x}$-matrix? Why?

In [58]:
# Create transformation matrix
def fd_matrix(T):
    D_T = np.zeros((T-1, T))
    return D_T

# Print the matrix
D_T = fd_matrix(T)
print(f'First differening matrix for T={T} \n', D_T)

First differening matrix for T=8 
 [[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]


In [59]:
# Transform the data.
y_diff = lm.perm(D_T, y.reshape(-1, 1))
x_diff = lm.perm(D_T, x.reshape(-1, x.shape[1]))

# Print x_diff
print(x_diff)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [60]:
# Check rank condition.
check_rank(x_diff)

Rank of demeaned x: 0
Eigenvalues of within-transformed x: [0. 0. 0. 0. 0. 0. 0. 0.]


Adjust `x_diff` such that the model can be estimated using the FD estimator. Adjust the labels to match with `x_diff`.

Estimate the model using the estimate() function, and print the results.

In [61]:
# Choose variables to include in fixed effects model
x_diff = x_diff[:, [0, 1, 2]]  # Experience, Experience^2, Union
label_x_fd = [
    'Experience',
    'Experience sqr',
    'Union'
]

In [62]:
# Estimate FE OLS using the demeaned variables.
fd_result = lm.estimate(y_diff, x_diff, transform='fd', N=N, T=T)

# Print results
lm.print_table((label_y, label_x_fd), fd_result, title='FD regression', floatfmt='.4f')

LinAlgError: Singular matrix

You should get a table that looks like this:

FD regression <br>
Dependent variable: Log wage

|                |    Beta |     Se |   t-values |
|----------------|---------|--------|------------|
| Experience     |  0.1158 | 0.0196 |     5.9096 |
| Experience sqr | -0.0039 | 0.0014 |    -2.8005 |
| Union          |  0.0428 | 0.0197 |     2.1767 |
| Married        |  0.0381 | 0.0229 |     1.6633 |
R² = 0.004 <br>
σ² = 0.196

**NB:** Did you use the right standard errors? Did you use the right number of time periods in the estimate() function?

How big is the union premium according to the estimate from this model? Compare the FD estimate with the estimate that you calculated from the FE regression. Is there a difference? If yes, what (if anything) can we conclude based on this finding?

### Using pandas

In [63]:
# first difference the data
pddat_diff = pddat.groupby("ID").diff().dropna() # groupby ID and take difference
pddat_diff = pddat_diff[desired_order[1:]] # reorder columns
datdiff = pddat_diff.to_numpy() # turn into numpy array

# Look at data
pddat_diff

,year,logwage,exp,expsq,union,mar,educ,hisp,black
1,1.0,0.655520,1.0,3.0,1.0,0.0,0.0,0.0,0.0
2,1.0,-0.508598,1.0,5.0,-1.0,0.0,0.0,0.0,0.0
3,1.0,0.088752,1.0,7.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.134912,1.0,9.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.131766,1.0,11.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
4355,1.0,0.759397,1.0,15.0,0.0,1.0,0.0,0.0,0.0
4356,1.0,-0.379336,1.0,17.0,1.0,0.0,0.0,0.0,0.0
4357,1.0,0.553419,1.0,19.0,-1.0,0.0,0.0,0.0,0.0
4358,1.0,-0.020068,1.0,21.0,1.0,0.0,0.0,0.0,0.0


In [64]:
# Select variables
y_delta = pddat_diff['logwage'].to_numpy().reshape(-1,1)
x_delta = pddat_diff[['exp','expsq','union','mar']].to_numpy()

In [65]:
# Estimate using the first differenced variables, y_delta and x_delta. 
fd_result = lm.estimate(y_delta, x_delta, transform='fd', N=N, T=T)

#print results
lm.print_table((label_y, label_x[1:5]), fd_result, title="First Differences", floatfmt='.4f')

First Differences
Dependent variable: Log wage

                   Beta      Se  t-values
--------------  -------  ------  -------------------------------------------------
Experience       0.1158  0.3876  [0.29860682 0.02129112 0.81830488 0.93393079]
Experience sqr  -0.0039  5.4365  [-0.01001557 -0.00071413 -0.02744676 -0.03132497]
Union            0.0428  0.1415  [0.11038216 0.00787042 0.30249228 0.34523423]
Married          0.0381  0.1239  [0.09838585 0.00701506 0.26961748 0.30771425]
R² = 0.004
σ² = 0.172


## Tests
### Test for serial correlation in the errors using an auxilliary AR(1) model
Tests assumption FD.3, where the errors $e_{it} = \Delta u_{it}$ should be serially uncorrelated.

We can easily test this assumption given the OLS residuals from the FD version of equation (1). Run the regression (note that you will lose data for
the first *two* periods)
\begin{equation}
\hat{e}_{it}=\rho\hat{e}_{it-1}+error_{it},\quad t=\color{red}{3},\dotsc,T,\quad i=1,\dotsc,N\tag{2}
\end{equation}

Do you find any evidence of serial correlation? Does FD.3 seem appropriate? Why don't we include an intercept in this auxilliary equation?

*Note:* Under FE.3, the idiosyncratic errors $u_{it}$
are uncorrelated. However, FE.3 implies that the $e_{it}$'s are autocorrelated. In fact, of the $u_{it}$'s are serially uncorrelated to begin with, corr $\left(e_{it},e_{it-1}\right)=-0.5$. (Check!) This test is of course only valid if the explanatory variables are strictly exogenous!

*Hint:* You can use the `perm` function to lag
the error term variable. Consider the following; 

$$
{\begin{bmatrix}
1 & 0 & 0 & \cdots & 0 & 0\\
0 & 1 & 0 & \cdots & 0 & 0\\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots\\
0 & 0 & 0 & \cdots & 1 & 0
\end{bmatrix}}_{T-1\times T}\times{\begin{bmatrix}y_{1}\\
y_{2}\\
\vdots\\
y_{T}
\end{bmatrix}}_{T \times 1}={\begin{bmatrix}y_{1}\\
y_{2}\\
\vdots\\
y_{T - 1}
\end{bmatrix}}_{T - 1\times 1}
$$

*Hint:* You can use the `perm` function to remove the first time-period in the residual. Consider the following; 

$$
{\begin{bmatrix}
0 & 1 & 0 & \cdots & 0 & 0\\
0 & 0 & 1 & \cdots & 0 & 0\\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots\\
0 & 0 & 0 & \cdots & 0 & 1
\end{bmatrix}}_{T-1\times T}\times{\begin{bmatrix}y_{1}\\
y_{2}\\
\vdots\\
y_{T}
\end{bmatrix}}_{T \times 1}={\begin{bmatrix}y_{2}\\
y_{3}\\
\vdots\\
y_{T}
\end{bmatrix}}_{T - 1\times 1}
$$

In [66]:
# Make function to calculate the serial correlation
def serial_corr(y, x, T):
    # Calculate the residuals
    b_hat = lm.estimate(y, x, N=N, T=T)['b_hat']
    e = y - x @ b_hat

    # Create a lag transformation matrix
    L_T = np.zeros((T-1, T))
    for t in range(1, T):
        L_T[t-1, t] = 1

    # Lag residuals
    e_l = lm.perm(L_T, e.reshape(-1, 1))

    # Create a transformation matrix that removes the first observation of each individual
    I_T = np.eye(T-1)

    
    # Remove first observation of each individual
    e = e[1:]

    # Calculate the serial correlation
    return lm.estimate(e, e_l, N=N, T=T)['b_hat']

In [67]:
# Estimate serial correlation
corr_result = serial_corr(y_diff, x_diff, T-1)

# Print results
label_ye = 'OLS residual, e\u1d62\u209c'
label_e = ['e\u1d62\u209c\u208B\u2081']
lm.print_table(
    (label_ye, label_e), corr_result, 
    title='Serial Correlation', floatfmt='.4f'
)

LinAlgError: Singular matrix

You should get a table that looks like this:

Serial Correlation <br>
Dependent variable: OLS residual, eᵢₜ

|       |    Beta |     Se |   t-values |
|-------|---------|--------|------------|
| eᵢₜ₋₁ | -0.3961 | 0.0147 |   -27.0185 |
R² = 0.182 <br>
σ² = 0.143

### Test for strict exogeneity

Add a lead of the union variable, $union_{i,t+1}$ to the equation (1) (note that you will lose data from period $T$ , 1987) and estimate the model with *fixed effects* (i.e., you have to demean $union_{i,t+1}$ along with all the other variables and throw out time constant variables). Is $union_{i,t+1}$ significant? What does this imply for the strict exogeneity assumption?

*Hint:* To lead a variable, think along the same lines as in the previous exercise.

In [70]:
# Lead union
F_T = np.zeros((T-1, T))

union_lead = lm.perm(F_T, x[:, 3].reshape(-1, 1))

In [69]:
# Remove the last observed year for every individual
I_T = np.eye(T-1)

x_exo = lm.perm(I_T, x)
y_exo = lm.perm(I_T, y)

IndexError: tuple index out of range

In [ ]:
# Add union_lead to x_exo
x_exo = np.hstack((x_exo, union_lead))

# Within transform the data
Q_T = # Fill in 
yw_exo = # Fill in 
xw_exo = # Fill in 

# Select variables
xw_exo = np.hstack((xw_exo[:, 1:5], xw_exo[:, -1].reshape(-1, 1)))

In [ ]:
# Estimate model
exo_test = # Fill in

# Print results
label_exo = label_x_fe + ['Union lead']
lm.print_table((label_y, label_exo), exo_test, title='Exogeneity test', floatfmt='.4f')

The table should look something like this:
Exogeneity test <br>
Dependent variable: Log wage

|                |    Beta |     Se |   t-values |
|----------------|---------|--------|------------|
| Experience     |  0.1213 | 0.0100 |    12.1001 |
| Experience sqr | -0.0050 | 0.0008 |    -6.3579 |
| Married        |  0.0436 | 0.0209 |     2.0898 |
| Union          |  0.0757 | 0.0218 |     3.4784 |
| Union lead     |  0.0515 | 0.0223 |     2.3063 |
R² = 0.146<br>
σ² = 0.128

### Time series dummies and FE
Add interactions on the form $d_{81}\cdot educ, d_{82}\cdot educ, ..., d_{87}\cdot educ$ and estimate the model with fixed effect. Has the return to education increased over time?

*Hint:* Remember that $educ_{i}$ doesn't vary over
time! Therefore we didn't use $educ$ in levels in the FE estimation.
However, if we suppose that the structural equation (4) contains a term $\sum_{s=2}^{T}\delta_{s}d_{s}educ_{i}$, it will be perfectly fine to within-transform these interactions since they vary over time (although in a highly structured manner - they equal
zero in all time periods but one, and then $educ$). Note that one
period is dropped for the within-transformation to work whereas the
levels term, $\beta_{5}educ_{i}$, is dropped to avoid producing a
constant row.

*Programming hint:* You want to append the dataset with a dummy matrix, that would look something like this:

$$
\begin{bmatrix}
0 & 0 & 0 & 0 & 0 & 0 & 0 \\
14 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 14 & 0 & 0 & 0 & 0 & 0 \\
\vdots & \vdots & \vdots & \vdots & \vdots & \ddots & \vdots \\
0 & 0 & 0 & 0 & 0 & 0 & 14 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 \\
9 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 9 & 0 & 0 & 0 & 0 & 0 \\
\vdots & \vdots & \ddots & \vdots & \vdots & \vdots & \vdots \\
0 & 0 & 0 & 0 & 0 & 0 & 9 \\
\end{bmatrix}
$$

This example shows our two first persons, that have 14 and 9 years of education respectively.Why is the first row for each person only zeros?

The matrix is constructed for you. Note how it can be done.

In [71]:
# This dummy block has a 0 row, as we need to exclude one
# year in order to not end up in the dummy trap.
dummy_block = np.eye(T, k=-1)[:, :-1]

# Expand thid dummy block to all persons
dummy_matrix = np.tile(dummy_block, (N, 1))

# We now create a n*t-1 matrix, with the person's education 
expanded_educ = np.transpose([x[:, 5]] * (T-1))

# We can now multiply the year dummy with a person's education
educ_dummies = dummy_matrix*expanded_educ

# We can now demean the variables
Q_T = demeaning_matrix(T)
educ_demean = lm.perm(Q_T, educ_dummies)
x_demean_dummies = np.hstack([x_demean, educ_demean])

# Add the year dummies to the label
label_x_interactions = label_x_fe + ['E81', 'E82', 'E83', 'E84', 'E85', 'E86', 'E87']

In [ ]:
# Estimate model
int_result = lm.estimate(y_demean, x_demean_dummies, transform='fe', N=N, T=T)

# Print results
lm.print_table((label_y, label_x_interactions), int_result, title='FE with year interactions', floatfmt='.4f')

LinAlgError: Singular matrix

You should get a table that looks like this:

FE with year interactions <br>
Dependent variable: Log wage

|                |    Beta |     Se |   t-values |
|----------------|---------|--------|------------|
| Experience     |  0.1705 | 0.0273 |     6.2462 |
| Experience sqr | -0.0060 | 0.0009 |    -6.9581 |
| Married        |  0.0475 | 0.0183 |     2.5925 |
| Union          |  0.0794 | 0.0193 |     4.1138 |
| E81            | -0.0010 | 0.0026 |    -0.4009 |
| E82            | -0.0062 | 0.0041 |    -1.5224 |
| E83            | -0.0114 | 0.0057 |    -2.0006 |
| E84            | -0.0136 | 0.0072 |    -1.8787 |
| E85            | -0.0162 | 0.0087 |    -1.8578 |
| E86            | -0.0170 | 0.0101 |    -1.6804 |
| E87            | -0.0167 | 0.0115 |    -1.4619 |
R² = 0.181 <br>
σ² = 0.123